In [6]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
from numpy.random import randn
import os
import seaborn as sns
import keras
from keras.models import load_model
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, Reshape, Input, Conv2DTranspose, Activation, LeakyReLU, BatchNormalization, Dropout, Resizing
from keras.losses import BinaryCrossentropy
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from imgaug import augmenters as iaa
import warnings
warnings.filterwarnings('ignore')

# Config
noise_dimension = 100
batch_size = 64
width, height, channels = 200, 200, 1

# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Path to the dataset
main_dir = '/content/drive/MyDrive/Healthy'

# Loading the dataset => resize images to 128*128*3
images = []
for i in os.listdir(main_dir):
    img_path = os.path.join(main_dir, i)
    print(f"Loading image: {img_path}")

    img = cv2.imread(img_path)

    # Check if the image was loaded successfully
    if img is None:
        print(f"Error loading image: {img_path}")
        continue

    img = cv2.resize(img, (200, 200))  # No need to convert to grayscale
    images.append(img)

dataset = np.array(images)

print(dataset.shape)

# Define the erasing transformation
eraser = iaa.CoarseDropout((0.02, 0.25), size_percent=(0.02, 0.25))

# Apply the transformation to each image in the dataset
augmented_images = []
for img in images:
    img = eraser.augment_image(img)
    augmented_images.append(img)

# Convert the list back to a numpy array
augmented_dataset = np.array(augmented_images)

print(augmented_dataset.shape)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading image: /content/drive/MyDrive/Healthy/image_027.png
Loading image: /content/drive/MyDrive/Healthy/image_020.png
Loading image: /content/drive/MyDrive/Healthy/image_024.png
Loading image: /content/drive/MyDrive/Healthy/image_019.png
Loading image: /content/drive/MyDrive/Healthy/image_012.png
Loading image: /content/drive/MyDrive/Healthy/image_014.png
Loading image: /content/drive/MyDrive/Healthy/image_032.png
Loading image: /content/drive/MyDrive/Healthy/image_011.png
Loading image: /content/drive/MyDrive/Healthy/image_022.png
Loading image: /content/drive/MyDrive/Healthy/image_008.png
Loading image: /content/drive/MyDrive/Healthy/image_005.png
Loading image: /content/drive/MyDrive/Healthy/image_001.png
Loading image: /content/drive/MyDrive/Healthy/image_033.png
Loading image: /content/drive/MyDrive/Healthy/image_002.png
Loading image: /content/drive/M

In [7]:
# Create a directory to save the images
save_dir = '/content/drive/MyDrive/Erasing_Transformation_Done'
os.makedirs(save_dir, exist_ok=True)

# Save each image
for i, img in enumerate(augmented_images):
    save_path = os.path.join(save_dir, f'image_{i}.png')
    cv2.imwrite(save_path, img)

print(f"Images saved in {save_dir}")


Images saved in /content/drive/MyDrive/Erasing_Transformation_Done


In [1]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
from numpy.random import randn
import os
import seaborn as sns
import keras
from keras.models import load_model
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, Reshape, Input, Conv2DTranspose, Activation, LeakyReLU, BatchNormalization, Dropout, Resizing
from keras.losses import BinaryCrossentropy
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from imgaug import augmenters as iaa
import warnings
warnings.filterwarnings('ignore')

# Config
noise_dimension = 100
batch_size = 64
width, height, channels = 200, 200, 1

# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Path to the dataset
main_dir = '/content/drive/MyDrive/Healthy'

# Loading the dataset => resize images to 128*128*3
images = []
failed_images = []
for i in os.listdir(main_dir):
    img_path = os.path.join(main_dir, i)
    print(f"Loading image: {img_path}")

    img = cv2.imread(img_path)

    # Check if the image was loaded successfully
    if img is None:
        print(f"Error loading image: {img_path}")
        failed_images.append(img_path)
        continue

    img = cv2.resize(img, (200, 200))  # No need to convert to grayscale
    images.append(img)

dataset = np.array(images)

print(f"Loaded {dataset.shape[0]} images successfully.")
print(f"Failed to load {len(failed_images)} images.")

# Define the erasing transformation
eraser = iaa.CoarseDropout((0.02, 0.25), size_percent=(0.02, 0.25))

# Apply the transformation to each image in the dataset
augmented_images = []
failed_transformations = []
for i, img in enumerate(images):
    try:
        img = eraser.augment_image(img)
        augmented_images.append(img)
    except Exception as e:
        print(f"Failed to transform image {i}: {str(e)}")
        failed_transformations.append(i)

# Convert the list back to a numpy array
augmented_dataset = np.array(augmented_images)

print(f"Transformed {augmented_dataset.shape[0]} images successfully.")
print(f"Failed to transform {len(failed_transformations)} images.")

# Create a directory to save the images
save_dir = '/content/drive/MyDrive/Erasing_Transformations'
os.makedirs(save_dir, exist_ok=True)

# Save each image
for i, img in enumerate(augmented_images):
    save_path = os.path.join(save_dir, f'image_{i}.png')
    cv2.imwrite(save_path, img)

print(f"Images saved in {save_dir}")


Mounted at /content/drive
Loading image: /content/drive/MyDrive/Healthy/image_027.png
Loading image: /content/drive/MyDrive/Healthy/image_020.png
Loading image: /content/drive/MyDrive/Healthy/image_024.png
Loading image: /content/drive/MyDrive/Healthy/image_019.png
Loading image: /content/drive/MyDrive/Healthy/image_012.png
Loading image: /content/drive/MyDrive/Healthy/image_014.png
Loading image: /content/drive/MyDrive/Healthy/image_032.png
Loading image: /content/drive/MyDrive/Healthy/image_011.png
Loading image: /content/drive/MyDrive/Healthy/image_022.png
Loading image: /content/drive/MyDrive/Healthy/image_008.png
Loading image: /content/drive/MyDrive/Healthy/image_005.png
Loading image: /content/drive/MyDrive/Healthy/image_001.png
Loading image: /content/drive/MyDrive/Healthy/image_033.png
Loading image: /content/drive/MyDrive/Healthy/image_002.png
Loading image: /content/drive/MyDrive/Healthy/image_031.png
Loading image: /content/drive/MyDrive/Healthy/image_010.png
Loading image: